Author: Roel Faber

Goal of this script: take the raw matchdata from Eredivisiedata.ipynb and use it to create useful features, such as match history, standings, form, etc.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set()

In [65]:
df = pd.read_csv('Data/matches.csv',index_col=0)
df.head()
df_20192020 = pd.read_csv('Data/matches_20192020.csv',index_col=0)

# Add last season of football to data

In [66]:
df = df[df['Season']!='2019-2020'].copy()

df = df.append(df_20192020).copy()

# Compute columns

In [67]:
df['GoalsScored'] = df['HomeGoals'] + df['AwayGoals']

In [68]:
df['GoalsDiff'] = df['HomeGoals'] - df['AwayGoals']

# Combine teams that have changed names

Evt. nog uitwerken: dan functie schrijven om te checken of bepaalde teams samen voorkomen in seizoenen.

In [69]:
teamlist = df['Home'].unique()

In [70]:
# Manually map teams with changed names to most recent name
rename_teams = {'Feijenoord':'Feyenoord',
               "Fortuna '54":'Fortuna Sittard', # Ook fusie met Sittardia, maar zaten gelijktijdig in Eredivisie
               'FSC Geleen':'Fortuna Sittard', 
               'DOS':'FC Utrecht',
                'USV Elinkwijk':'FC Utrecht',
                'Velox':'FC Utrecht',
               'Sportclub Enschede':'FC Twente',
               "VV Alkmaar '54":'AZ Alkmaar',
               "AZ '67 Alkmaar":'AZ Alkmaar',
               'GVAV Groningen':'FC Groningen',
               'FC Den Haag-ADO':'ADO Den Haag',
               'FC Den Haag':'ADO Den Haag',
               "Drechtsteden '79":'FC Dordrecht',
               "Dordrecht '90":'FC Dordrecht',
               "S.V.V./Dordrecht '90":'FC Dordrecht',
               "FC Den Bosch '67":'FC Den Bosch',
               'BVV Den Bosch':'FC Den Bosch',
               } 

In [71]:
df = df.replace(rename_teams)

# Get more informative features

Features to include:

1) Standings in previous season(s)

2) Recent results against the opponent

3) Recent form (results of previous matches)

## Standings in seasons

In [72]:
seasons = df['Season'].unique()
roundnrs = [l for l in range(1,35)]

index_iter = [seasons,roundnrs]

In [73]:
mult_index = pd.MultiIndex.from_product(index_iter, names=['Season', 'Round'])

In [74]:
standings_df = pd.DataFrame(columns=mult_index,index=df['Home'].unique())
points_df = pd.DataFrame(columns=mult_index,index=df['Home'].unique())

In [12]:
def get_standings(seasondf,seasonname):
    """Get the final results table for a season
    
    Parameters
    -------------
    seasondf, pd.DataFrame
        Dataframe containing the matches in a season
        
    seasonname, str
        string of season, should match value in seasondf
        
    Output
    -------------
    
    standings_df, pd.DataFrame
        Dataframe with standings based on points at the end of a season
    points_df, pd.DataFrame
        Dataframe with points acquired in a given season"""
    teamlist = seasondf['Home'].unique()
    standings = pd.DataFrame(index=teamlist,columns=range(1,35))
    for index, row in seasondf.iterrows():
        standings.loc[row['Home'],row['Round']] = row['HomePoints']
        standings.loc[row['Away'],row['Round']] = row['AwayPoints']
    standings['Total'] = standings.sum(axis=1)
    standings['Total'].rank(ascending=False).sort_values()
    standings['Final'] = standings['Total'].rank(ascending=False).sort_values()
    for index,row in standings.iterrows():
        standings_df.at[index,seasonname] = row['Final']
        points_df.at[index,seasonname] = row['Total']

In [58]:
test_season.head()

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,...,Home_standings_last_season,Home_standings_this_season,Home_standings_form,Home_points_last_season,Home_points_this_season,Away_standings_last_season,Away_standings_this_season,Away_standings_form,Away_points_last_season,Away_points_this_season
0,2019-2020,1,PEC Zwolle,Willem II,1,3,2,0,3,4,...,13.0,15.5,NaN,NaN,NaN,10.0,5.0,NaN,NaN,NaN
1,2019-2020,1,Vitesse,AFC Ajax,2,2,3,1,1,4,...,5.5,6.5,NaN,NaN,NaN,1.0,1.5,NaN,NaN,NaN
2,2019-2020,1,FC Emmen,FC Groningen,0,1,2,0,3,1,...,14.0,12.0,NaN,NaN,NaN,8.5,9.0,NaN,NaN,NaN
3,2019-2020,1,VVV-Venlo,RKC Waalwijk,3,1,1,3,0,4,...,11.5,13.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
4,2019-2020,1,FC Twente,PSV,1,1,3,1,1,2,...,NaN,14.0,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN


In [96]:
def get_points_round(seasondf,seasonname):
    teamlist = seasondf['Home'].unique()
    for index, row in seasondf.iterrows():
        points_df.at[row['Home'],(seasonname,row['Round'])] = row['HomePoints']
        points_df.at[row['Away'],(seasonname,row['Round'])] = row['AwayPoints']

In [76]:
test_season = df[df['Season']=='2019-2020']

In [97]:
%%time
for season in df['Season'].unique():
    seasondf = df[df['Season']==season]
    get_points_round(seasondf,season)

Wall time: 2.32 s


In [98]:
cumpoints_df = points_df.transpose().fillna(0).groupby(level=0).cumsum().transpose()

In [100]:
cumpoints_df

Season           1959-1960                                    ... 2019-2020  \
Round                   1  2  3   4   5   6   7   8   9   10  ...        25   
ADO Den Haag             1  4  4   4   4   7   7  10  10  13  ...        19   
AFC Ajax                 3  4  7   8   8  11  14  17  20  23  ...        53   
Blauw Wit                3  3  6   9   9  12  12  12  13  13  ...         0   
FC Utrecht               3  3  6   7   7  10  13  14  15  16  ...        38   
Feyenoord                0  1  2   3   6   9  10  11  12  15  ...        47   
Fortuna Sittard          0  3  3   6   6   7   7   7   8  11  ...        25   
Rapid JC Heerlen         0  1  1   4   5   5   6   6   9  10  ...         0   
FC Volendam              3  3  6   9   9   9  12  15  16  16  ...         0   
VVV-Venlo                3  3  4   4   5   8  11  12  13  13  ...        28   
DOS Utrecht              3  6  9  10  13  13  13  13  16  16  ...         0   
DWS Amsterdam            0  0  0   0   3   4   4   4   7   7  ...         0   
MVV Maastricht           0  0  0   0   1   1   1   4   4   7  ...         0   
NAC Breda                0  1  1   4   7   8  11  14  17  20  ...         0   
PSV                      3  4  7   8   9  10  13  16  19  22  ...        46   
FC Twente                0  3  6   6   9   9  12  12  12  12  ...        27   
Sittardia                0  1  1   4   7   8   8   8   8   8  ...         0   
Sparta Rotterdam         3  6  6   6   9   9   9  12  12  12  ...        33   
Willem II                1  4  7   8   8   9  12  13  13  16  ...        44   
AZ Alkmaar               0  0  0   0   0   0   0   0   0   0  ...        53   
Groninger VAV            0  0  0   0   0   0   0   0   0   0  ...         0   
NOAD Tilburg             0  0  0   0   0   0   0   0   0   0  ...         0   
De Volewijckers          0  0  0   0   0   0   0   0   0   0  ...         0   
Heracles Almelo          0  0  0   0   0   0   0   0   0   0  ...        33   
Go Ahead Eagles          0  0  0   0   0   0   0   0   0   0  ...         0   
Telstar                  0  0  0   0   0   0   0   0   0   0  ...         0   
FC Groningen             0  0  0   0   0   0   0   0   0   0  ...        35   
FC Xerxes                0  0  0   0   0   0   0   0   0   0  ...         0   
NEC Nijmegen             0  0  0   0   0   0   0   0   0   0  ...         0   
Xerxes/D.H.C.            0  0  0   0   0   0   0   0   0   0  ...         0   
Holland Sport            0  0  0   0   0   0   0   0   0   0  ...         0   
SVV Schiedam             0  0  0   0   0   0   0   0   0   0  ...         0   
HFC Haarlem              0  0  0   0   0   0   0   0   0   0  ...         0   
SBV Excelsior            0  0  0   0   0   0   0   0   0   0  ...         0   
FC Den Bosch             0  0  0   0   0   0   0   0   0   0  ...         0   
Vitesse                  0  0  0   0   0   0   0   0   0   0  ...        38   
FC Amsterdam             0  0  0   0   0   0   0   0   0   0  ...         0   
Roda JC Kerkrade         0  0  0   0   0   0   0   0   0   0  ...         0   
De Graafschap            0  0  0   0   0   0   0   0   0   0  ...         0   
WVV Wageningen           0  0  0   0   0   0   0   0   0   0  ...         0   
SC Eindhoven             0  0  0   0   0   0   0   0   0   0  ...         0   
PEC Zwolle               0  0  0   0   0   0   0   0   0   0  ...        25   
FC Wageningen            0  0  0   0   0   0   0   0   0   0  ...         0   
SBV Haarlem              0  0  0   0   0   0   0   0   0   0  ...         0   
PEC Zwolle '82           0  0  0   0   0   0   0   0   0   0  ...         0   
Helmond Sport            0  0  0   0   0   0   0   0   0   0  ...         0   
FC Dordrecht             0  0  0   0   0   0   0   0   0   0  ...         0   
BV Veendam               0  0  0   0   0   0   0   0   0   0  ...         0   
RKC Waalwijk             0  0  0   0   0   0   0   0   0   0  ...        15   
sc Heerenveen            0  0  0   0   0   0   0  

In [ ]:
def get_standings(seasondf,seasonname):
    """Get the final results table for a season
    
    Parameters
    -------------
    seasondf, pd.DataFrame
        Dataframe containing the matches in a season
        
    seasonname, str
        string of season, should match value in seasondf
        
    Output
    -------------
    
    standings_df, pd.DataFrame
        Dataframe with standings based on points at the end of a season
    points_df, pd.DataFrame
        Dataframe with points acquired in a given season"""
    teamlist = seasondf['Home'].unique()
    
    
    
    standings = pd.DataFrame(index=teamlist,columns=range(1,35))
    for index, row in seasondf.iterrows():
        standings.loc[row['Home'],row['Round']] = row['HomePoints']
        standings.loc[row['Away'],row['Round']] = row['AwayPoints']
    standings['Total'] = standings.sum(axis=1)
    standings['Total'].rank(ascending=False).sort_values()
    standings['Final'] = standings['Total'].rank(ascending=False).sort_values()
    for index,row in standings.iterrows():
        standings_df.at[index,seasonname] = row['Final']
        points_df.at[index,seasonname] = row['Total']

In [13]:
%%time
for season in df['Season'].unique():
    seasondf = df[df['Season']==season]
    get_standings(seasondf,season)

Wall time: 6.44 s


In [68]:
standings_df.to_csv('Data/final_standings.csv')
points_df.to_csv('Data/final_points.csv')

In [69]:
standings_df = pd.read_csv('Data/final_standings.csv', index_col=0)
points_df = pd.read_csv('Data/final_points.csv', index_col=0)

## Add information to matches df

In [17]:
df['Home_standings_last_season'] = np.nan
df['Home_standings_this_season'] = np.nan
df['Home_standings_form'] = np.nan
df['Home_points_last_season'] = np.nan
df['Home_points_this_season'] = np.nan
df['Away_standings_last_season'] = np.nan
df['Away_standings_this_season'] = np.nan
df['Away_standings_form'] = np.nan
df['Away_points_last_season'] = np.nan
df['Away_points_this_season'] = np.nan

In [18]:
df.head()

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,...,Home_standings_last_season,Home_standings_this_season,Home_standings_form,Home_points_last_season,Home_points_this_season,Away_standings_last_season,Away_standings_this_season,Away_standings_form,Away_points_last_season,Away_points_this_season
0,1959-1960,1,ADO Den Haag,Willem II,3,3,3,1,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1959-1960,1,AFC Ajax,NAC Breda,3,0,1,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1959-1960,1,Blauw Wit,Sittardia,5,2,1,3,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1959-1960,1,FC Utrecht,MVV Maastricht,2,0,1,3,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1959-1960,1,Feyenoord,Sparta Rotterdam,0,1,2,0,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add standings

In [32]:
standings_df.head()

,1959-1960,1960-1961,1961-1962,1962-1963,1963-1964,1964-1965,1965-1966,1966-1967,1967-1968,1968-1969,...,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020
ADO Den Haag,8,9,14.5,12,7.5,3.5,3,4,4,6,...,7,15,9,9,13.5,11.5,11,7,8.5,17
AFC Ajax,1,2,4,2.5,6,12,1,1,1,2,...,1,1,1,1,2,2,2,2,1,1.5
Blauw Wit,7,NaN,3,10,15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FC Utrecht,15,15,NaN,NaN,NaN,NaN,14,17,NaN,NaN,...,9,11,5,10,11.5,5,4,5,5.5,6.5
Feyenoord,3,1,1.5,4,3.5,1,2,2,2,1,...,10,2,2.5,2,4,3,1,4,3,3


In [30]:
for index, row in df.iterrows():
    hometeam = row['Home']
    awayteam = row['Away']
    season = row['Season']
    startyear,endyear = season.split('-')
    lastseason = f"{int(startyear)-1}-{int(endyear)-1}"
    
    # Voor laatste seizoen zijn deze df's wel prima
    try:
        df.at[index,'Home_standings_last_season'] = standings_df.at[hometeam,lastseason]
        df.at[index,'Away_standings_last_season'] = standings_df.at[awayteam,lastseason]
    except KeyError: # Voor eerste seizoen
        continue
    # Nog aanpassen: punten en standings in multi-index df stoppen
    df.at[index,'Home_standings_this_season'] = standings_df.at[hometeam,season]
    df.at[index,'Away_standings_this_season'] = standings_df.at[awayteam,season]

In [31]:
df

,Season,Round,Home,Away,HomeGoals,AwayGoals,Result,HomePoints,AwayPoints,GoalsScored,...,Home_standings_last_season,Home_standings_this_season,Home_standings_form,Home_points_last_season,Home_points_this_season,Away_standings_last_season,Away_standings_this_season,Away_standings_form,Away_points_last_season,Away_points_this_season
0,1959-1960,1,ADO Den Haag,Willem II,3,3,3,1,1,6,...,13.0,15.5,NaN,NaN,NaN,10.0,5.0,NaN,NaN,NaN
1,1959-1960,1,AFC Ajax,NAC Breda,3,0,1,3,0,3,...,5.5,6.5,NaN,NaN,NaN,1.0,1.5,NaN,NaN,NaN
2,1959-1960,1,Blauw Wit,Sittardia,5,2,1,3,0,7,...,14.0,12.0,NaN,NaN,NaN,8.5,9.0,NaN,NaN,NaN
3,1959-1960,1,FC Utrecht,MVV Maastricht,2,0,1,3,0,2,...,11.5,13.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
4,1959-1960,1,Feyenoord,Sparta Rotterdam,0,1,2,0,3,1,...,NaN,14.0,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2019-2020,26,sc Heerenveen,AFC Ajax,1,3,2,0,3,4,...,11.5,10.5,NaN,NaN,NaN,1.0,1.5,NaN,NaN,NaN
228,2019-2020,26,FC Utrecht,Sparta Rotterdam,5,1,1,3,0,6,...,5.5,6.5,NaN,NaN,NaN,NaN,10.5,NaN,NaN,NaN
229,2019-2020,26,Feyenoord,Willem II,2,0,1,3,0,2,...,3.0,3.0,NaN,NaN,NaN,10.0,5.0,NaN,NaN,NaN
230,2019-2020,26,Heracles Almelo,RKC Waalwijk,4,2,1,3,0,6,...,7.0,8.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN


## Results from last 5 matches

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18247 entries, 0 to 231
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 18247 non-null  object 
 1   Round                  18247 non-null  int64  
 2   Home                   18247 non-null  object 
 3   Away                   18247 non-null  object 
 4   HomeGoals              18247 non-null  int64  
 5   AwayGoals              18247 non-null  int64  
 6   Result                 18247 non-null  int64  
 7   HomePoints             18247 non-null  int64  
 8   AwayPoints             18247 non-null  int64  
 9   GoalsScored            18247 non-null  int64  
 10  HomeAdvantage          18247 non-null  int64  
 11  standings_last_season  0 non-null      float64
 12  standings_this_season  0 non-null      float64
 13  standings_form         0 non-null      float64
 14  points_last_season     0 non-null      float64
 15  poin

In [28]:
def get_form(df, team, window, outputcol):
    teamdf = df.loc[(df.Home==team) | (df.Away==team)]
    pointslist = []
    for index, row in df.iterrows():
        if row.Home==team:
            pointslist.append(row.HomePoints)
        elif row.Away==team:
            pointslist.append(row.AwayPoints)
    teamdf.loc[:,'TeamPoints'] = pointslist
    teamdf.loc[:,outputcol] = teamdf.TeamPoints.rolling(window=window).sum()
    df.loc[teamdf[teamdf.Home==team].index,'Home_'+outputcol] = teamdf.loc[:,outputcol]
    df.loc[teamdf[teamdf.Away==team].index,'Away_'+outputcol] = teamdf.loc[:,outputcol]

In [29]:
get_form(df, 'AFC Ajax', 5, 'RecentForm')

C:\Users\roel_\anaconda3\envs\football-predictions\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\roel_\anaconda3\envs\football-predictions\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


ValueError: cannot reindex from a duplicate axis

In [ ]:
# get_form_opponent(df, team, opponent, window, outputcol):
#     Optie 1
#     ---------------
#     establish recentdf
#     pointslist
#     toevoegen aan df
#     berekenen rolling som
#     toewijzen aan alle combinaties
    
#     Optie 2
#     ----------------
#     Itereer met functie over rijen heen
    
    
    

In [ ]:
team = 'AFC Ajax'
opponent = 'PSV'

In [ ]:
teamdf = df.loc[((df.Home==team)&]

In [ ]:
teamlist = df.Home.unique()

In [ ]:
%%time
for team in teamlist:
    get_form(df, team, 5, 'RecentForm')

In [ ]:
df = df.drop('RecentForm',axis=1)

In [ ]:
df.to_csv('Data/matcheswithform.csv')

In [ ]:
df = pd.read_csv('Data/matcheswithform.csv', index_col = 0)

In [ ]:
df